In [4]:
import os

In [5]:
%pwd

'/Users/davidibrahim/Programming/mlops1/Project1/research'

In [6]:
%cd ..

/Users/davidibrahim/Programming/mlops1/Project1


/Users/davidibrahim/Programming/mlops1/Project1/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
%pwd

'/Users/davidibrahim/Programming/mlops1/Project1'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str



In [13]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories
from src.datascience import logger


In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))
        self.schema = read_yaml(str(schema_filepath))

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [11]:
import os
import pandas as pd
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib

In [20]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


    

In [24]:
try:
    logger.info("Starting the model training process.")
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    
    # Log the configuration loaded
    logger.info(f"Model Trainer Config: {model_trainer_config}")

    model_trainer = ModelTrainer(config=model_trainer_config)
    logger.info("Training the model.")
    model_trainer.train()
    logger.info("Model training completed successfully.")
except Exception as e:
    logger.error(f"An error occurred during the model training process: {e}")
    raise e


[2025-07-05 11:00:32,251 - datascience - INFO - Starting the model training process.]
[2025-07-05 11:00:32,255 - datascience - INFO - YAML file config/config.yaml loaded successfully.]
[2025-07-05 11:00:32,256 - datascience - INFO - YAML file params.yaml loaded successfully.]
[2025-07-05 11:00:32,258 - datascience - INFO - YAML file schema.yaml loaded successfully.]
[2025-07-05 11:00:32,260 - datascience - INFO - Directory artifacts created or already exists.]
[2025-07-05 11:00:32,261 - datascience - INFO - Directory artifacts/model_trainer created or already exists.]
[2025-07-05 11:00:32,261 - datascience - INFO - Model Trainer Config: ModelTrainerConfig(root_dir='artifacts/model_trainer', train_data_path='artifacts/data_transformation/train.csv', test_data_path='artifacts/data_transformation/test.csv', model_name='model.joblib', alpha=0.2, l1_ratio=0.1, target_column='quality')]
[2025-07-05 11:00:32,262 - datascience - INFO - Training the model.]
[2025-07-05 11:00:32,276 - datascienc